implement an Oxygen Add-on(Add-on-UI.jpb) to mark up the selected string in opened xml by the prompt of a custom Oxygen AI Positron Action(ref_markup_optV2.json), then you may replace the original string diretly; preview the difference by compare side by side(Add-on-UI-preview.jpb); or just copy the results. You may configure the endpoint by local flask API with key, select model from local Ollama(Add-on-UI-setting.jpb and output of localLlmWIntelGPUV2.ipynb).

please follow the development guide from Oxygen:
https://www.oxygenxml.com/doc/versions/27.1/ug-editor/topics/extending-oxygen.html
https://www.oxygenxml.com/doc/versions/27.1/ug-editor/topics/extend-oxygen-with-plugins.html
https://www.oxygenxml.com/doc/versions/27.1/ug-editor/topics/packing-and-deploying-plugins-as-addons.html
https://www.oxygenxml.com/doc/versions/27.1/ug-editor/dev_guide/introduction-dev-guide.html

Okay, I can help you outline the development of an Oxygen XML Add-on that integrates with your local LLM setup via the AI Positron Assistant.

Here's a step-by-step solution:

1.  **Create a new Jupyter Notebook** named `Oxygen-AI-Markup-add-on.ipynb` to document the add-on development.
2.  **Define the Add-on Structure**: Describe the necessary files and folders for an Oxygen Add-on.
3.  **Detail `plugin.xml`**: Explain the role of `plugin.xml` and provide a basic template.
4.  **Outline Java Implementation**: Sketch the Java classes required to:
    *   Create toolbar/menu actions.
    *   Build the UI for interacting with the user (input, preview, copy, replace).
    *   Communicate with the local Flask API, leveraging Oxygen's AI Positron Enterprise settings for endpoint, API key, and model selection.
5.  **Describe UI Components**: Based on your images, list the UI elements needed.
6.  **Build and Deployment Notes**: Briefly touch upon packaging the add-on.
7.  **Update `localLlmWIntelGPUV2.ipynb`**: Add a cell to link to the new add-on development notebook.

### [Oxygen-AI-Markup-add-on.ipynb](file:///c%3A/Project/OCR/Oxygen-AI-Markup-add-on.ipynb)

Create a new notebook file with the following cells to outline the Oxygen Add-on development.



In [ ]:
<VSCode.Cell language="markdown">
# Oxygen XML Add-on for AI-Powered Markup

This notebook outlines the development of an Oxygen XML Add-on that integrates with a local LLM service (via Flask API) to provide AI-powered markup capabilities. The add-on will allow users to:

1.  Select text in an XML document.
2.  Send the selected text and a predefined prompt (from a custom AI Positron Action configuration like `ref_markup_optV2.json`) to the local LLM.
3.  Preview the LLM's suggested markup alongside the original selection.
4.  Replace the selected text with the AI-generated markup.
5.  Copy the AI-generated markup to the clipboard.
6.  Configure the connection to the local LLM service using Oxygen's AI Positron Enterprise settings (address, API key, model).

This development will follow Oxygen's guidelines for creating add-ons and plugins.

**References:**
*   [Extending Oxygen XML Editor](https://www.oxygenxml.com/doc/versions/27.1/ug-editor/topics/extending-oxygen.html)
*   [Extend Oxygen XML Editor with Plugins](https://www.oxygenxml.com/doc/versions/27.1/ug-editor/topics/extend-oxygen-with-plugins.html)
*   [Packing and Deploying Plugins as Add-ons](https://www.oxygenxml.com/doc/versions/27.1/ug-editor/topics/packing-and-deploying-plugins-as-addons.html)
*   [Oxygen XML Editor Developer Guide](https://www.oxygenxml.com/doc/versions/27.1/ug-editor/dev_guide/introduction-dev-guide.html)
*   [AI Positron Custom Connector](https://www.oxygenxml.com/doc/ug-addons/topics/ai-positron-custom-connector.html)
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 1. Add-on Structure

An Oxygen XML Add-on typically has the following structure:

```
Oxygen-AI-Markup-Add-on/
├── addon.xml                 # Add-on descriptor
├── plugin.xml                # Plugin descriptor (core logic)
├── lib/                      # Folder for JAR dependencies (e.g., HTTP client, JSON parser)
│   └── custom-ai-markup.jar  # Our plugin's compiled JAR
├── icons/                    # Folder for custom icons
│   └── ai_markup_icon.png
└── ... (other resources)
```

*   **`addon.xml`**: Describes the add-on for Oxygen's add-on manager.
*   **`plugin.xml`**: Defines the plugin's extensions, actions, and entry points.
*   **`lib/custom-ai-markup.jar`**: The compiled Java code containing the add-on's logic.
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 2. `plugin.xml` Configuration

This file is the heart of the plugin, defining how it integrates with Oxygen.

Key sections:
*   **Plugin Information**: ID, name, version, description, vendor.
*   **Extension Points**:
    *   `ro.sync.exml.plugin.workspace.WorkspaceAccessPluginExtension`: To add custom actions to menus/toolbars.
    *   Possibly others for accessing editor content or AI Positron settings.
*   **Libraries**: Specifies the JAR file containing the plugin code.
</VSCode.Cell>
<VSCode.Cell language="xml">
<!-- plugin.xml example -->
<!DOCTYPE plugin SYSTEM "../plugin.dtd">
<plugin
    id="com.example.oxygen.ai.markup"
    name="AI Markup Assistant"
    description="Provides AI-powered markup suggestions using a local LLM."
    version="1.0.0"
    vendor="Example Corp"
    class="com.example.oxygen.ai.markup.AIMarkupPluginExtension">

    <runtime>
        <library name="lib/custom-ai-markup.jar"/>
        <!-- Add other necessary libraries e.g. for HTTP requests, JSON parsing -->
        <!-- <library name="lib/httpclient.jar"/> -->
        <!-- <library name="lib/json.jar"/> -->
    </runtime>

    <extension type="WorkspaceAccessPluginExtension" class="com.example.oxygen.ai.markup.AIMarkupWorkspaceAccessPluginExtension"/>
    
    <!-- Potentially an extension to read AI Positron Enterprise settings -->
    <!-- Or use Oxygen's API to fetch these settings directly -->

</plugin>
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 3. Java Implementation Outline

The core logic will be implemented in Java.

**Key Java Classes:**

1.  **`AIMarkupPluginExtension.java`**:
    *   Implements `ro.sync.exml.plugin.PluginExtension`.
    *   Handles plugin lifecycle (initialization, cleanup).

2.  **`AIMarkupWorkspaceAccessPluginExtension.java`**:
    *   Implements `ro.sync.exml.plugin.workspace.WorkspaceAccessPluginExtension`.
    *   `applicationStarted(StandalonePluginWorkspaceAccess pluginWorkspaceAccess)`: This is where UI elements like toolbar buttons or menu items are added.
    *   `applicationClosing()`: For cleanup.

3.  **`MarkupAction.java`** (extends `javax.swing.AbstractAction`):
    *   Triggered by the toolbar button/menu item.
    *   Gets the selected text from the current editor.
    *   Retrieves AI Positron Enterprise settings (Base URL, API Key, Model) using Oxygen's API.
        *   `PluginWorkspaceProvider.getPluginWorkspace().getOptionsStorage().getOption(OptionTags.AI_POSITRON_ENTERPRISE_CONNECTOR_URL, "")`
        *   `PluginWorkspaceProvider.getPluginWorkspace().getOptionsStorage().getOption(OptionTags.AI_POSITRON_ENTERPRISE_API_KEY, "")`
        *   `PluginWorkspaceProvider.getPluginWorkspace().getOptionsStorage().getOption(OptionTags.AI_POSITRON_ENTERPRISE_MODEL, "")`
        *   (Note: Exact `OptionTags` need to be verified from Oxygen's API or documentation for AI Positron Enterprise settings).
    *   Loads the prompt from `ref_markup_optV2.json` (or a similar configuration mechanism). This JSON could be bundled with the add-on or user-configurable.
    *   Constructs the request payload for the local Flask API.
    *   Calls `LocalLlmApiService.callLLM()`.
    *   Opens `MarkupPreviewDialog` with the original text and AI suggestion.

4.  **`MarkupPreviewDialog.java`** (extends `javax.swing.JDialog`):
    *   Displays the original selected text and the AI-generated markup side-by-side (as shown in `Add-on-UI-preview.jpb`).
    *   Provides "Apply", "Copy", and "Cancel" buttons.
    *   "Apply": Replaces the selection in the editor with the AI markup.
    *   "Copy": Copies the AI markup to the clipboard.

5.  **`LocalLlmApiService.java`**:
    *   Handles HTTP communication with the local Flask API (`http://127.0.0.1:5000/ai/chat/completions`).
    *   Sends the prompt, selected text, model, and API key.
    *   Parses the JSON response.
    *   Uses a standard Java HTTP client (e.g., `java.net.http.HttpClient`) or a library like Apache HttpClient.

**Example Snippet for `MarkupAction.java` (Conceptual):**
</VSCode.Cell>
<VSCode.Cell language="java">
// Conceptual snippet for MarkupAction.java
// ... imports ...

// Inside actionPerformed(ActionEvent e):
// StandalonePluginWorkspaceAccess workspaceAccess = PluginWorkspaceProvider.getPluginWorkspace();
// WXMLEditor editor = (WXMLEditor) workspaceAccess.getCurrentEditorAccess(StandalonePluginWorkspace.MAIN_EDITING_AREA);
// if (editor != null && editor.hasSelection()) {
//     String selectedText = editor.getSelectedText();
//     
//     // Retrieve AI Positron Enterprise Settings
//     OptionsStorage optionsStorage = workspaceAccess.getOptionsStorage();
//     String positronBaseURL = optionsStorage.getOption("AI_POSITRON_ENTERPRISE_CONNECTOR_URL", "http://127.0.0.1:5000/ai/"); // Key needs verification
//     String positronApiKey = optionsStorage.getOption("AI_POSITRON_ENTERPRISE_API_KEY", ""); // Key needs verification
//     String positronModel = optionsStorage.getOption("AI_POSITRON_ENTERPRISE_MODEL", "gemma3:1b"); // Key needs verification
//
//     // Load prompt from ref_markup_optV2.json (simplified)
//     // In a real scenario, parse the JSON file.
//     String contextPrompt = "You are an experienced Buddhist Tripitaka expert..."; // Loaded from JSON
//     String fullPrompt = contextPrompt.replace("${contextInfo}", "") + "\n\nSelected text: " + selectedText;
//
//     // Construct messages for the API
//     // List<Message> messages = new ArrayList<>();
//     // messages.add(new Message("system", contextPrompt.replace("${contextInfo}", "")));
//     // messages.add(new Message("user", selectedText));
//
//     // String aiSuggestion = LocalLlmApiService.callLLM(positronBaseURL, positronApiKey, positronModel, messages);
//     // if (aiSuggestion != null) {
//     //     MarkupPreviewDialog dialog = new MarkupPreviewDialog(workspaceAccess.getParentFrame(), selectedText, aiSuggestion, editor);
//     //     dialog.setVisible(true);
//     // } else {
//     //     // Handle error
//     // }
// }
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 4. Interacting with AI Positron Settings

The add-on needs to read the configuration that the user sets in Oxygen's AI Positron Enterprise settings dialog:
*   **AI Connector**: Should be "Custom AI service".
*   **Address**: The base URL for the local Flask API (e.g., `http://127.0.0.1:5000/ai/`).
*   **API Key**: The key generated by your Flask API key manager.
*   **Model**: The specific Ollama model to use (e.g., `gemma3:1b`, `gemma3:4b`).

Oxygen's `ro.sync.exml.workspace.api.options.OptionsStorage` API should be used to retrieve these values. The exact option keys (e.g., `OptionTags.AI_POSITRON_ENTERPRISE_CONNECTOR_URL`) need to be identified from Oxygen's API documentation or by inspecting how AI Positron itself stores these settings.

The prompt itself will be based on the structure defined in `ref_markup_optV2.json`. The add-on will need to:
1.  Load this JSON file (it could be bundled with the add-on or placed in a known location).
2.  Extract the `context` and `examples`.
3.  Format the `context` by injecting the selected text from the editor.
4.  Construct the `messages` array for the `/ai/chat/completions` endpoint, similar to how the Flask app does it.
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 5. UI Design

Based on the provided images:

*   **Main Action UI (`Add-on-UI.jpb` like functionality):**
    *   A toolbar button and/or a menu item (e.g., under "Tools" or "AI Positron").
    *   Icon: A custom icon (e.g., `ai_markup_icon.png`).
    *   Clicking this action will:
        *   Get the current selected text in the XML editor.
        *   Fetch AI Positron Enterprise settings.
        *   Fetch the prompt from `ref_markup_optV2.json`.
        *   Call the local LLM service.
        *   Open the preview dialog.

*   **Preview Dialog (`Add-on-UI-preview.jpb` like functionality):**
    *   A modal dialog.
    *   Two main text areas:
        *   "Current" or "Original Selection": Displays the text selected by the user (read-only).
        *   "Proposed Changes" or "AI Suggestion": Displays the XML fragment returned by the LLM (editable or read-only, depending on design).
    *   Buttons:
        *   "Apply": Replaces the selection in the editor with the content of the "Proposed Changes" area.
        *   "Copy": Copies the content of the "Proposed Changes" area to the clipboard.
        *   "Cancel" / "Close": Closes the dialog without making changes.
    *   The side-by-side comparison is crucial for usability. Oxygen provides diff utilities that might be leveraged, or simple JTextAreas can be used.

*   **Settings (`Add-on-UI-setting.jpb` like functionality):**
    *   This functionality is already provided by Oxygen's "Plugins / Oxygen AI Positron Assistant Enterprise / AI Service Configuration" dialog. The add-on will **read** these settings, not provide its own settings UI for the endpoint, API key, and model.
    *   The add-on might have its own settings dialog for things like:
        *   Path to the `ref_markup_optV2.json` or similar prompt configuration files.
        *   Default behavior (e.g., auto-copy after generation).
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 6. Build and Deployment

1.  **Compile Java Code**: Use a Java compiler (javac) or an IDE (like Eclipse or IntelliJ IDEA with Oxygen SDK) to compile the Java source files into `.class` files.
2.  **Create JAR File**: Package the compiled `.class` files and any resources (like the `ref_markup_optV2.json` if bundled) into `custom-ai-markup.jar`.
3.  **Prepare Add-on Directory**:
    *   Create the directory structure as outlined in Section 1.
    *   Place `custom-ai-markup.jar` into the `lib/` folder.
    *   Create `addon.xml` and `plugin.xml`.
4.  **Package Add-on**: Zip the entire `Oxygen-AI-Markup-Add-on/` directory. This zip file is the add-on package.
5.  **Install Add-on**: In Oxygen, go to "Help" > "Install new add-ons..." and select the packaged zip file.

**`addon.xml` Example:**
</VSCode.Cell>
<VSCode.Cell language="xml">
<!-- addon.xml -->
<addon id="com.example.oxygen.ai.markup.addon"
       name="AI Markup Assistant Add-on"
       version="1.0.0"
       vendor="Example Corp"
       description="Integrates with a local LLM for AI-powered XML markup suggestions.">
    
    <script type="plugin" file="plugin.xml"/>
    
    <licenseDialog file="license.txt"/> <!-- Optional -->
    <readmeDialog file="readme.html"/> <!-- Optional -->
    
    <javaVersion min="11"/> <!-- Specify minimum Java version -->
    <oxygenVersion min="27.0"/> <!-- Specify minimum Oxygen version -->
</addon>
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 7. Next Steps

1.  **Set up Java Development Environment**: Ensure JDK and an IDE (optional, but recommended) are set up.
2.  **Integrate Oxygen SDK**: If using an IDE, configure it with the Oxygen SDK for access to Oxygen-specific APIs.
3.  **Implement Core Java Classes**: Start with `AIMarkupPluginExtension` and `AIMarkupWorkspaceAccessPluginExtension`.
4.  **Develop UI**: Create the `MarkupPreviewDialog`.
5.  **Implement API Service**: Write the `LocalLlmApiService` to communicate with your Flask backend.
6.  **Handle Prompt Configuration**: Decide how `ref_markup_optV2.json` will be accessed (bundled, user path).
7.  **Thorough Testing**: Test with various XML files, selections, and LLM responses.
8.  **Error Handling**: Implement robust error handling for API calls, file access, etc.
</VSCode.Cell>

```

### c:\Project\OCR\localLlmWIntelGPUV2.ipynb

Add a new markdown cell to reference the Oxygen Add-on development notebook.

````xml
<!-- filepath: c:\Project\OCR\localLlmWIntelGPUV2.ipynb -->
// ...existing code...
<VSCode.Cell id="9f621c9c" language="markdown">


## Summary

**Yes, you can load `gemma3:4b` (3.3GB) using MX250's shared memory access**, but expect:

- ✅ **Works**: Model will load and run
- ⚠️ **Slower**: 2-3x slower than VRAM-only models
- ⚠️ **System Impact**: Uses significant system RAM
- ✅ **Better Quality**: Better responses than smaller models

**Best Practice**: Use `gemma3:1b` for speed, `gemma3:4b` for quality when you can tolerate slower responses.
</VSCode.Cell>
<VSCode.Cell language="markdown">
# Oxygen XML Add-on Development

For details on developing an Oxygen XML Add-on to integrate with this local LLM service for AI-powered markup, please refer to the following notebook:

[Oxygen AI Markup Add-on Development](./Oxygen-AI-Markup-add-on.ipynb)

This add-on will leverage the AI Positron Enterprise custom connector settings to communicate with the Flask API defined in this notebook.
</VSCode.Cell>